### 协程
"to yield"在字典中有两个意思 产出与让步  对于python生成器中的yield也成立 <br>
yield item这行代码会产出一个值 提供给next的调用方 <br>
还会做出让步 暂停执行生成器 让调用方继续工作 直到需要使用另一个值的时候再调用next()。

#### 协程的四个状态

In [1]:
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine recived', x)

In [7]:
my_coro = simple_coroutine()

In [8]:
my_coro

<generator object simple_coroutine at 0x10ede1eb8>

In [9]:
next(my_coro)  #首先调用next启动协程 相对于send(None) 在yield语句处暂停

-> coroutine started


In [10]:
my_coro.send(42)

-> coroutine recived 42


StopIteration: 

send方法会成为暂停的yield表达式的值 所以仅当协程处于暂停状态才能调用send方法<br>
最先调用的next(my_coro)函数这一步通常成为**预激**协程 让协程向前执行到第一个yield表达式

In [11]:
def simple_coro2(a):
    print('-> Start: a = ', a)
    b = yield a
    print('-> Recived b = ', b)
    c = yield a + b
    print('-> Recived c= ', c)

In [12]:
from inspect import getgeneratorstate

In [17]:
my_coro2 = simple_coro2(14)

In [18]:
getgeneratorstate(my_coro2) ## 等待开始执行状态

'GEN_CREATED'

In [19]:
next(my_coro2) ## 在yield处暂停

-> Start: a =  14


14

In [21]:
getgeneratorstate(my_coro2) ## 在yield表达式处暂停状态

'GEN_SUSPENDED'

In [22]:
my_coro2.send(28)

-> Recived b =  28


42

In [23]:
my_coro2.send(99)

-> Recived c=  99


StopIteration: 

In [24]:
getgeneratorstate(my_coro2) ##执行结束状态

'GEN_CLOSED'

** 关键的一点是协程在yield关键字所在位置暂停执行。前面说过，在赋值语句中 = 右边的代码在赋值之前就执行 **
#### 使用协程计算移动平均值

In [25]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

#### 预激协程的装饰器

In [26]:
from functools import wraps

In [30]:
def coroutine(func):
    """装饰器 向前执行到第一个yield表达式 预激func"""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [31]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [33]:
getgeneratorstate(averager()) ##已经预激协程

'GEN_SUSPENDED'

使用yield from调用协程的时候会自动预激协程

#### 使用 yield from
yield from可以简化for循环中的yield表达式
有时间看cookbook中的这个[章节](http://python3-cookbook.readthedocs.io/zh_CN/latest/c04/p14_flattening_nested_sequence.html)<br>
yield from x表达式对x对象所做的第一件事是调用iter(x)从中获取迭代器 所以x可以是任意可迭代对象

In [34]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i

In [35]:
list(gen())

['A', 'B', 1, 2]

In [36]:
def gen():
    yield from 'AB'
    yield from range(1,3)

In [37]:
list(gen())

['A', 'B', 1, 2]

** yield from的功能是打开双向通道 把最外层的调用方与最内层的子生成器连接起来 这样二者可以直接发送和产出值 还可以直接传入异常 二不用在位于中间的协程中添加大量处理异常的样板代码 **

In [40]:
from collections import namedtuple

In [41]:
Result = namedtuple('Result', 'count average')

In [42]:
# 子生成器
def averager():
    total = 0.0
    count = 0 
    average= None
    while True:
        term = yield average
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [43]:
# 委派生成器
def grouper(results, key):
    while True:
        results[key] = yield from averager()

In [49]:
# 客户端代码 即调用方
def client(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
    report(results)

In [46]:
# 输出报告
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))

In [47]:
data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

In [50]:
client(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


### 使用asyncio包处理并发

In [2]:
import asyncio
import itertools
import sys

@asyncio.coroutine
def spin(msg):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' +msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            yield from asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    write('' * len(status) + '\x08' * len(status))

@asyncio.coroutine
def slow_function():
    yield from asyncio.sleep(.1)
    return 42

@asyncio.coroutine
def supervisor():
    spinner = asyncio.async(spin('thinking!'))
    print('spinner object:', spinner)
    result = yield from slow_function()
    spinner.cancel()
    return result

def main():
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(supervisor())
    loop.close()
    print('Answer:', result)

In [ ]:
main()